In [1]:
import numpy as np
from torch import nn
from torch.utils.data import DataLoader, sampler
from torch.utils.tensorboard import SummaryWriter
import torch
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
log = False

In [2]:
log = True
writer = SummaryWriter('./log')

In [3]:
from src.utils.data import getPandas, writePandas
data = getPandas('data')
test = data.sample(frac=0.2, random_state=10)
data = data.drop(test.index)
validate = data.sample(frac=0.1, random_state=10)
train = data.drop(validate.index)
test = test.reset_index(drop=True)
train = train.reset_index(drop=True)
validate = validate.reset_index(drop=True)
writePandas('data_test', test)
writePandas('data_train', train)
writePandas('data_validate', validate)

In [2]:
from src.dl.loader import RegressDataset
from src.dl.resnet import RegressCNN
from sklearn.model_selection import KFold
import collections
train_set = RegressDataset('train')
val_set = RegressDataset('validate')
test_set = RegressDataset('test')
fold_num = 5
kf = KFold(n_splits=fold_num, shuffle=True, random_state=10)
best_models = np.empty(dtype=collections.OrderedDict, shape=fold_num)
net = RegressCNN().cuda()
loss_fn = nn.SmoothL1Loss().cuda()
#lr = 1e-2
#optim = torch.optim.Adam(net.parameters(), lr=lr)
lr = 1e-3
#optim = torch.optim.SGD([{'params': net.fc.weight, 'lr': 1e-2}], lr=lr)
optim = torch.optim.SGD(net.parameters(), lr=lr)
epoch = 100
train_loader = DataLoader(train_set, batch_size=8)
val_loader = DataLoader(val_set, batch_size=8)
for i in range(epoch):
    total_loss = 0
    predy = np.array([])
    y = np.array([])
    net.train()
    for step, [img, labels, score] in enumerate(train_loader):
        img = img.cuda()
        labels = labels.cuda()
        score = score.cuda()
        output = net(img, labels).squeeze(-1)
        loss = loss_fn(output, score)

        optim.zero_grad()
        loss.backward()
        optim.step()
        #scheduler.step()
        
        output = output.cpu().detach().numpy()
        score = score.cpu().detach().numpy()
        predy = np.concatenate((predy, output), axis=0)
        y = np.concatenate((y, score), axis=0)
        total_loss += loss.item()
        
    nmse = np.sum((y - predy) ** 2) / np.sum(y ** 2)
    mse = np.sum((y - predy) ** 2) / len(train_set)
    rmse = np.sqrt(mse)
    mae = np.sum(np.abs(y - predy)) / len(train_set)
    meany = np.mean(y)
    rsquared = r2_score(y, predy)
    print('train epoch {}, loss {}, nmse {}, rmse {}, mae {}, rsquared {}'.format(i, total_loss, nmse, rmse, mae, rsquared))
    
    if log:
        writer.add_scalar('train/Loss', total_loss, i)
        writer.add_scalar('train/NMSE', nmse, i)
        writer.add_scalar('train/RMSE', rmse, i)
        writer.add_scalar('train/MAE', mae, i)
        writer.add_scalar('train/rsquared', rsquared, i)
        
    net.eval()
    total_loss = 0
    predy = np.array([])
    y = np.array([])
    with torch.no_grad():
        total = 0
        correct = 0
        for step, [img, labels, score] in enumerate(val_loader):
            img = img.cuda()
            labels = labels.cuda()
            score = score.cuda()
            output = net(img, labels).squeeze(-1)
            loss = loss_fn(output, score)

            output = output.cpu().detach().numpy()
            score = score.cpu().detach().numpy()
            #print("y: ", score)
            #print("y_pred: ", output)
            predy = np.concatenate((predy, output), axis=0)
            y = np.concatenate((y, score), axis=0)
            total_loss += loss.item()

    nmse = np.sum((y - predy) ** 2) / np.sum(y ** 2)
    mse = np.sum((y - predy) ** 2) / len(val_set)
    rmse = np.sqrt(mse)
    mae = np.sum(np.abs(y - predy)) / len(val_set)
    meany = np.mean(y)
    rsquared = r2_score(y, predy)
    print('validation epoch {}, loss {}, nmse {}, rmse {}, mae {}, rsquared {}'.format(i, total_loss, nmse, rmse, mae, rsquared))
    if log:
        writer.add_scalar('validation/Loss', total_loss, i)
        writer.add_scalar('validation/NMSE', nmse, i)
        writer.add_scalar('validation/RMSE', rmse, i)
        writer.add_scalar('validation/MAE', mae, i)
        writer.add_scalar('validation/rsquared', rsquared, i)

In [3]:
from src.dl.loader import BlockRegressDataset
from src.dl.resnet import RegressResNet3d
from sklearn.model_selection import KFold
import collections
train_set = BlockRegressDataset('train')
val_set = BlockRegressDataset('validate')
test_set = BlockRegressDataset('test')
fold_num = 5
kf = KFold(n_splits=fold_num, shuffle=True, random_state=10)
best_models = np.empty(dtype=collections.OrderedDict, shape=fold_num)
net = RegressResNet3d().cuda()
loss_fn = nn.SmoothL1Loss().cuda()
#lr = 1e-2
#optim = torch.optim.Adam(net.parameters(), lr=lr)
lr = 1e-3
#optim = torch.optim.SGD([{'params': net.fc.weight, 'lr': 1e-2}], lr=lr)
optim = torch.optim.SGD(net.parameters(), lr=lr)
epoch = 100
train_loader = DataLoader(train_set, batch_size=8)
val_loader = DataLoader(val_set, batch_size=8)
for i in range(epoch):
    total_loss = 0
    predy = np.array([])
    y = np.array([])
    net.train()
    for step, [img, labels, score] in enumerate(train_loader):
        img = img.cuda()
        labels = labels.cuda()
        score = score.cuda()
        output = net(img, labels).squeeze(-1)
        loss = loss_fn(output, score)

        optim.zero_grad()
        loss.backward()
        optim.step()
        #scheduler.step()
        
        output = output.cpu().detach().numpy()
        score = score.cpu().detach().numpy()
        predy = np.concatenate((predy, output), axis=0)
        y = np.concatenate((y, score), axis=0)
        total_loss += loss.item()
        
    nmse = np.sum((y - predy) ** 2) / np.sum(y ** 2)
    mse = np.sum((y - predy) ** 2) / len(train_set)
    rmse = np.sqrt(mse)
    mae = np.sum(np.abs(y - predy)) / len(train_set)
    meany = np.mean(y)
    rsquared = r2_score(y, predy)
    print('train epoch {}, loss {}, nmse {}, rmse {}, mae {}, rsquared {}'.format(i, total_loss, nmse, rmse, mae, rsquared))
    
    if log:
        writer.add_scalar('train/Loss', total_loss, i)
        writer.add_scalar('train/NMSE', nmse, i)
        writer.add_scalar('train/RMSE', rmse, i)
        writer.add_scalar('train/MAE', mae, i)
        writer.add_scalar('train/rsquared', rsquared, i)
        
    net.eval()
    total_loss = 0
    predy = np.array([])
    y = np.array([])
    with torch.no_grad():
        total = 0
        correct = 0
        for step, [img, labels, score] in enumerate(val_loader):
            img = img.cuda()
            labels = labels.cuda()
            score = score.cuda()
            output = net(img, labels).squeeze(-1)
            loss = loss_fn(output, score)

            output = output.cpu().detach().numpy()
            score = score.cpu().detach().numpy()
            #print("y: ", score)
            #print("y_pred: ", output)
            predy = np.concatenate((predy, output), axis=0)
            y = np.concatenate((y, score), axis=0)
            total_loss += loss.item()

    nmse = np.sum((y - predy) ** 2) / np.sum(y ** 2)
    mse = np.sum((y - predy) ** 2) / len(val_set)
    rmse = np.sqrt(mse)
    mae = np.sum(np.abs(y - predy)) / len(val_set)
    meany = np.mean(y)
    rsquared = r2_score(y, predy)
    print('validation epoch {}, loss {}, nmse {}, rmse {}, mae {}, rsquared {}'.format(i, total_loss, nmse, rmse, mae, rsquared))
    if log:
        writer.add_scalar('validation/Loss', total_loss, i)
        writer.add_scalar('validation/NMSE', nmse, i)
        writer.add_scalar('validation/RMSE', rmse, i)
        writer.add_scalar('validation/MAE', mae, i)
        writer.add_scalar('validation/rsquared', rsquared, i)

train epoch 0, loss 1.2705457803388311, nmse 0.5820618636310471, rmse 0.3204060718967204, mae 0.2597217988753498, rsquared -0.9319191856627904
validation epoch 0, loss 0.07125460409419046, nmse 0.4535196613114828, rmse 0.21779763927523607, mae 0.1830128508782032, rsquared -0.23640535663170903
train epoch 1, loss 1.0372686503228414, nmse 0.475816910917891, rmse 0.28969173277289395, mae 0.23332297877917935, rsquared -0.579281991317262
validation epoch 1, loss 0.06166337387403477, nmse 0.3884365399300048, rmse 0.2015650110663673, mae 0.1690533409210856, rsquared -0.05897287295576126
train epoch 2, loss 0.8905752562248191, nmse 0.4092871860630559, rmse 0.2686768444095969, mae 0.21535403693768734, rsquared -0.3584634496898822
validation epoch 2, loss 0.057590275256412395, nmse 0.35900168599532234, rmse 0.1937775016764472, mae 0.1598619233008839, rsquared 0.021273727536202824
train epoch 3, loss 0.7962586899699644, nmse 0.3665646991316259, rmse 0.2542678697417324, mae 0.20400058945563615, rs

KeyboardInterrupt: 

In [5]:
test_loader = DataLoader(test_set, batch_size=64)
net.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for step, [img, labels, score] in enumerate(test_loader):
        img = img.cuda()
        labels = labels.cuda()
        score = score.cuda()
        #score = torch.Tensor(np.random.randint(0, 3, score.size())).type(torch.LongTensor).cuda()
        output = net(img, labels).squeeze(-1)
        loss = loss_fn(output, score)
        
        predy = np.concatenate((predy, output.cpu().detach().numpy()), axis=0)
        y = np.concatenate((y, score.cpu().detach().numpy()), axis=0)
        total_loss += loss.item()
nmse = np.sum((y - predy) ** 2) / np.sum(y ** 2)
mse = np.sum((y - predy) ** 2) / len(test_set)
rmse = np.sqrt(mse)
mae = np.sum(np.abs(y - predy)) / len(test_set)
meany = np.mean(y)
rsquared = 1 - np.sum((predy - y) ** 2) / np.sum((y - meany) ** 2)
print('epoch {}, loss {}, nmse {}, rmse {}, mae {}, rsquared {}'.format(i, total_loss, nmse, rmse, mae, rsquared))

epoch 99, loss 14.81796445565536, nmse 0.38246806504802155, rmse 0.2794655588970451, mae 0.2638362386154233, rsquared -0.5403551487274227


In [ ]:
loss_fn = nn.SmoothL1Loss().cuda()
lr = 1e-3
optim = torch.optim.SGD([
    {'params': net.fc.weight, 'lr': 1e-2}
    ], lr=lr, momentum=0.3)
#optim = torch.optim.SGD(net.parameters(), lr=1e-1, momentum=0.3)
#scheduler = torch.optim.lr_scheduler.ExponentialLR(optim, gamma=0.99)
#optim = torch.optim.Adam(net.parameters(), lr=lr, betas=[0.3, 0.1])
epoch = 100
dataset = BlockRegressDataset('train')
kf = KFold(n_splits=5, shuffle=True, random_state=10)
for fold, (train_idx, val_idx) in enumerate(kf.split(np.arange(len(dataset)))):
    train_sampler = sampler.SubsetRandomSampler(train_idx)
    val_sampler = sampler.SubsetRandomSampler(val_idx)
    train_loader = DataLoader(dataset, batch_size=64, sampler=train_sampler)
    val_loader = DataLoader(dataset, batch_size=64, sampler=val_sampler)
    for i in range(epoch):
        total_loss = 0
        predy = np.array([])
        y = np.array([])
        net.train()
        for step, [img, labels, score] in enumerate(train_loader):
            img = img.cuda()
            labels = labels.cuda()
            score = score.cuda()
            output = net(img, labels).squeeze(-1)
            loss = loss_fn(output.float(), score.float())

            optim.zero_grad()
            loss.backward()
            optim.step()
            #scheduler.step()
        
        net.eval()
        with torch.no_grad():
            for step, [img, labels, score] in enumerate(val_loader):
                img = img.cuda()
                labels = labels.cuda()
                score = score.cuda()
                output = net(img, labels).squeeze(-1)
                loss = loss_fn(output.float(), score.float())
        
                predy = np.concatenate((predy, output.cpu().detach().numpy()), axis=0)
                y = np.concatenate((y, score.cpu().detach().numpy()), axis=0)

                total_loss += loss.item()

        nmse = np.sum((y - predy) ** 2) / np.sum(y ** 2)
        mse = np.sum((y - predy) ** 2) / len(train_idx)
        mae = np.sum(np.abs(y - predy)) / len(train_idx)
        meany = np.mean(y)
        rsquared = np.sum((predy - meany) ** 2) / np.sum((y - meany) ** 2)
        print('epoch {}, loss {}, nmse {}, mae {}, rsquared {}'.format(i, total_loss, nmse, mae, rsquared))
        writer.add_scalar('Loss', total_loss, i)
        writer.add_scalar('NMSE', nmse, i)
        writer.add_scalar('MAE', mae, i)
        writer.add_scalar('rsquared', rsquared, i)

writer.close()

In [ ]:
predy = []
y = []
for fold, (train_idx, val_idx) in enumerate(kf.split(np.arange(len(dataset)))):
    train_sampler = sampler.SubsetRandomSampler(train_idx)
    val_sampler = sampler.SubsetRandomSampler(val_idx)
    train_loader = DataLoader(dataset, batch_size=64, sampler=train_sampler)
    val_loader = DataLoader(dataset, batch_size=64, sampler=val_sampler)
    net.eval()
    with torch.no_grad():
        for step, [img, labels, score] in enumerate(train_loader):
            img = img.cuda()
            labels = labels.cuda()
            score = score.cuda()
            output = net(img, labels).squeeze(-1)
            loss = loss_fn(output.float(), score.float())
    
            predy = np.concatenate((predy, output.cpu().detach().numpy()), axis=0)
            y = np.concatenate((y, score.cpu().detach().numpy()), axis=0)

            total_loss += loss.item()

    nmse = np.sum((y - predy) ** 2) / np.sum(y ** 2)
    mse = np.sum((y - predy) ** 2) / len(train_idx)
    mae = np.sum(np.abs(y - predy)) / len(train_idx)
    meany = np.mean(y)
    rsquared = np.sum((predy - meany) ** 2) / np.sum((y - meany) ** 2)
    print(rsquared)
    print(mse)
    print(mae)
    break